In [2]:
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings("ignore")

# Pre-preprocessing
 - (пришлось отредактировать 14 строку 3 листа, там странный формат id)

In [3]:
for i in range(1, 4):
    pd.read_excel("Taxonomy examples (edited).xlsx", sheet_name=i - 1, header=None).to_csv(f"tree{i}.csv", index=False)

In [4]:
def strip_csv(file_path):
    """
    Auxiliary function for specific dataset.
    """
    with open(file_path, 'r') as file:
        stripped_lines = [line[1:-2] for line in file]

    with open(file_path, 'w') as file:
        file.write('\n'.join(stripped_lines[1:]))
        
for file in ['tree1.csv', 'tree2.csv', 'tree3.csv']:
    strip_csv(file)

In [5]:
naming = ["id"]+list(range(10))
df1 = pd.read_csv('tree1.csv', names=naming)
df2 = pd.read_csv('tree2.csv', names=naming)
df3 = pd.read_csv('tree3.csv', names=naming)
df1.head()

,id,0,1,2,3,4,5,6,7,8,9
0,3.2.1.,NaN,NaN,Data mining,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,3.2.1.1.,NaN,NaN,NaN,Data cleaning,NaN,NaN,NaN,NaN,NaN,NaN
2,3.2.1.2.,NaN,NaN,NaN,Collaborative filtering,NaN,NaN,NaN,NaN,NaN,NaN
3,3.2.1.2.1*,NaN,NaN,NaN,NaN,Item-based,NaN,NaN,NaN,NaN,NaN
4,3.2.1.2.2*,NaN,NaN,NaN,NaN,Scalable,NaN,NaN,NaN,NaN,NaN


In [6]:
reg = r"[*.]+(?!.*[*.])" # Matches tracing symbol (either ., * or **), did not find interpretation of it
df1["id"] = df1["id"].str.replace(reg, "")
df2["id"] = df2["id"].str.replace(reg, "")
df3["id"] = df3["id"].str.replace(reg, "")

# DF to directed graph

In [7]:
!pip install networkx

In [8]:
import networkx as nx
import re

def df_to_graph(data):
    """
    Auxiliary function to transform dataframe into nx.DiGraph
    """
    G = nx.DiGraph()
    for r in data.iterrows():
        index = str(r[1].id)
        name = r[1][~r[1].isna()].iloc[1]
        G.add_node(name)
        
        parent_index = index[:-len(re.findall(r"\d+(?!.*\d+)", index)[0])-1]
        parent = data[data.id == parent_index]
        if parent.shape[0] == 0:
            continue

            
        parent = parent.iloc[0]
        parent_name = parent[~parent.isna()].values[1]  
        
        if parent_name in G:
            G.add_edge(parent_name, name)
            
    return G
    
tree1 = df_to_graph(df1)
tree2 = df_to_graph(df2)
tree3 = df_to_graph(df3)

In [9]:
tree1.nodes

NodeView(('Data mining', 'Data cleaning', 'Collaborative filtering', 'Item-based', 'Scalable', 'Association rules', 'Types of association rules', 'Interestingness', 'Parallel computation', 'Clustering', 'Massive data clustering', 'Consensus clustering', 'Fuzzy clustering', 'Additive clustering', 'Feature weight clustering', 'Conceptual clustering', 'Biclustering', 'Nearest-neighbor search', 'Data stream mining', 'Graph mining', 'Graph partitioning', 'Frequent graph mining', 'Graph based conceptual clustering', 'Anomaly detection', 'Critical nodes detection', 'Process mining', 'Text mining', 'Text categorization', 'Key-phrase indexing', 'Data mining tools', 'Sequence mining', 'Rule and pattern discovery', 'Trajectory clustering', 'Market graph', 'Formal concept analysis'))

In [10]:
tree2.edges

OutEdgeView([('Data mining', 'Data cleaning'), ('Data mining', 'Collaborative filtering'), ('Data mining', 'Association rules'), ('Data mining', 'Clustering'), ('Data mining', 'Nearest-neighbor search'), ('Data mining', 'Other  mining'), ('Data mining', 'Formal concept analysis'), ('Collaborative filtering', 'Item-based'), ('Collaborative filtering', 'Scalable'), ('Association rules', 'Types of association rules'), ('Association rules', 'Interestingness'), ('Association rules', 'Parallel computation'), ('Clustering', 'Massive data clustering'), ('Clustering', 'Consensus clustering'), ('Clustering', 'Fuzzy clustering'), ('Clustering', 'Additive clustering'), ('Clustering', 'Feature weight clustering'), ('Clustering', 'Conceptual clustering'), ('Clustering', 'Biclustering'), ('Other  mining', 'Data stream mining'), ('Other  mining', 'Graph mining'), ('Other  mining', 'Process mining'), ('Other  mining', 'Text mining'), ('Other  mining', 'Data mining tools'), ('Other  mining', 'Sequence m

In [11]:
# def get_leaves(graph, node): # Redundant function
#     leaves = []
#     for u, children in nx.dfs_successors(graph, node).items():
#         for v in children:
#             if graph.out_degree(v) == 0:
#                 leaves.append(v)
#     return leaves

# get_leaves(tree2, 'Sequence mining')

In [12]:
def least_common_ancestor(graph: nx.DiGraph, nodes: list):
    """
    Least common ancestor: the 'deepest' node in graph, containing all nodes as successors.
    """
    ancestors = [set(nx.ancestors(graph, node)) for node in nodes]
    common_ancestors = set.intersection(*ancestors)
    
    return min(common_ancestors, key=lambda x: len(nx.shortest_path(graph, x, nodes[0])) - 1) # rework

least_common_ancestor(tree2, ['Rule and pattern discovery', 'Trajectory clustering', 'Types of association rules'])

'Data mining'

In [16]:
def map_tree(treeG: nx.DiGraph, treeS: nx.DiGraph):
    """
    Least common ancestor mapping. (Assumes presence of all nodes from treeG in treeS.)
    
    Maps every node X from treeG to least common ancestor node of all successors of X from treeG in treeS (unless it does not have any, in which case maps it to itself).
    """
    mappings = []
    for node in treeG.nodes:
        if treeG.in_degree(node) != 0 and treeG.out_degree(node) != 0:
            successors = list(nx.dfs_successors(treeG, node).values())[0]
            lca = least_common_ancestor(treeS, successors)

            mappings += [(node, lca)]
            print(f"{node} ---> {lca}")
        else:
            mappings += [(node, node)]
            print(f"{node} ---> {node}")
    
    return mappings
            
        
print(map_tree(tree1, tree3))

Data mining ---> Data mining
Data cleaning ---> Data cleaning
Collaborative filtering ---> Collaborative filtering
Item-based ---> Item-based
Scalable ---> Scalable
Association rules ---> Association rules
Types of association rules ---> Types of association rules
Interestingness ---> Interestingness
Parallel computation ---> Parallel computation
Clustering ---> Clustering
Massive data clustering ---> Massive data clustering
Consensus clustering ---> Consensus clustering
Fuzzy clustering ---> Fuzzy clustering
Additive clustering ---> Additive clustering
Feature weight clustering ---> Feature weight clustering
Conceptual clustering ---> Conceptual clustering
Biclustering ---> Biclustering
Nearest-neighbor search ---> Nearest-neighbor search
Data stream mining ---> Data stream mining
Graph mining ---> Graph mining
Graph partitioning ---> Graph partitioning
Frequent graph mining ---> Frequent graph mining
Graph based conceptual clustering ---> Graph based conceptual clustering
Anomaly det

In [17]:
import ete3

In [19]:
def convert_to_ete(initial: nx.DiGraph):
    """
    Auxiliary function that converts nx.DiGraph to an ete3 Tree in order to simplify visualization.
    """
    subtrees = {node:ete3.Tree(name=node) for node in initial.nodes()}
    [*map(lambda edge:subtrees[edge[0]].add_child(subtrees[edge[1]]), initial.edges())]
    
    return subtrees[list(tree1.nodes)[0]]

print(convert_to_ete(tree1).get_ascii())


           /-Data cleaning
          |
          |                       /-Item-based
          |-Collaborative filtering
          |                       \-Scalable
          |
          |                 /-Types of association rules
          |                |
          |-Association rules-Interestingness
          |                |
          |                 \-Parallel computation
          |
          |          /-Massive data clustering
          |         |
          |         |--Consensus clustering
          |         |
          |         |--Fuzzy clustering
          |         |
          |-Clustering-Additive clustering
          |         |
          |         |--Feature weight clustering
          |         |
          |         |--Conceptual clustering
          |         |
          |          \-Biclustering
          |
          |--Nearest-neighbor search
-Data mining
          |--Data stream mining
          |
          |            /-Graph partitioning
          

In [20]:
print(convert_to_ete(tree2).get_ascii())


           /-Data cleaning
          |
          |                       /-Item-based
          |-Collaborative filtering
          |                       \-Scalable
          |
          |                 /-Types of association rules
          |                |
          |-Association rules-Interestingness
          |                |
          |                 \-Parallel computation
          |
          |          /-Massive data clustering
          |         |
          |         |--Consensus clustering
          |         |
          |         |--Fuzzy clustering
          |         |
          |-Clustering-Additive clustering
          |         |
          |         |--Feature weight clustering
          |         |
          |         |--Conceptual clustering
          |         |
          |          \-Biclustering
          |
          |--Nearest-neighbor search
-Data mining
          |             /-Data stream mining
          |            |
          |            |    

In [21]:
print(convert_to_ete(tree3).get_ascii())


           /-Data cleaning
          |
          |                       /-Item-based
          |-Collaborative filtering
          |                       \-Scalable
          |
          |                 /-Types of association rules
          |                |
          |-Association rules-Interestingness
          |                |
          |                 \-Parallel computation
          |
          |                                 /-Massive data clustering
          |                                |
          |          /Clustering feature data-Fuzzy clustering
          |         |                      |
          |         |                       \-Conceptual clustering
          |         |
          |         |--Feature weight clustering
          |-Clustering
          |         |                          /-Additive clustering
          |         |-Clustering similarity data
          |         |                          \-Consensus clustering
          |         |
 

In [20]:
test1 = pd.read_csv('test1.csv', names=["id"]+list(range(3)))
test2 = pd.read_csv('test2.csv', names=["id"]+list(range(3)))
test1

,id,0,1,2
0,1.,root,NaN,NaN
1,1.1.,NaN,left,NaN
2,1.2.,NaN,right,NaN
3,1.1.1.,NaN,NaN,human
4,1.1.2.,NaN,NaN,horse
5,1.2.1.,NaN,NaN,chimp


In [21]:
test2

,id,0,1,2
0,1.,root,NaN,NaN
1,1.1.,NaN,left,NaN
2,1.2.,NaN,right,NaN
3,1.1.1.,NaN,NaN,human
4,1.1.2.,NaN,NaN,chimp
5,1.2.1.,NaN,NaN,horse


In [23]:
test1_tree = df_to_graph(test1)
test2_tree = df_to_graph(test2)

In [24]:
map_tree(test1_tree, test2_tree)

root ---> root
left ---> root
right ---> left
human ---> human
horse ---> horse
chimp ---> chimp
